In [19]:
from __future__ import print_function
import sys
import os
import numpy as np
import torch
import pandas as pd

from pytorch_edgeml.graph.protoNN import ProtoNN
from pytorch_edgeml.trainer.protoNNTrainer import ProtoNNTrainer
import pytorch_edgeml.utils as utils
import helpermethods as helper

## Paul's Data
It is assumed that the USPS data has already been downloaded and set up with the help of `fetch_usps.py` and is placed in the `./usps10` subdirectory.

In [20]:
# Load data
DATA_DIR = './desktop'
#train, test = np.load(DATA_DIR + '/train.npy'), np.load(DATA_DIR + '/test.npy')
train = np.genfromtxt(DATA_DIR + '/_train.csv', delimiter=",", skip_header=1)
test = np.genfromtxt(DATA_DIR + '/_test.csv', delimiter=",", skip_header=1)
x_train, y_train = train[:, 1:], train[:, 0]
x_test, y_test = test[:, 1:], test[:, 0]

numClasses = max(y_train) - min(y_train) + 1
numClasses = max(numClasses, max(y_test) - min(y_test) + 2) # +2 to allow for zero index
numClasses = int(numClasses)

y_train = helper.to_onehot(y_train, numClasses)
y_test = helper.to_onehot(y_test, numClasses)

print(x_train.shape)
print(y_train.shape)
# Load data
train = np.genfromtxt(DATA_DIR + '/_train.csv', delimiter=",", skip_header=1)
test = np.genfromtxt(DATA_DIR + '/_test.csv', delimiter=",", skip_header=1)
x_train, y_train = train[:, 1:], train[:, 0]
x_test, y_test = test[:, 1:], test[:, 0]
# Convert y to one-hot
minval = 0
#numClasses = int(10)
y_train = helper.to_onehot(y_train, numClasses, minlabel=minval)
y_test = helper.to_onehot(y_test, numClasses, minlabel=minval)
dataDimension = x_train.shape[1]
print(x_train.shape)
print(y_train.shape)


dataDimension = x_train.shape[1]
numClasses = y_train.shape[1]

(216126, 124)
(216126, 10)
(216126, 124)
(216126, 10)


## Model Parameters

Note that ProtoNN is very sensitive to the value of the hyperparameter $\gamma$, here stored in valiable GAMMA. If GAMMA is set to None, median heuristic will be used to estimate a good value of $\gamma$ through the helper.getGamma() method. This method also returns the corresponding W and B matrices which should be used to initialize ProtoNN (as is done here).

In [21]:
PROJECTION_DIM = 10
NUM_PROTOTYPES = 24
REG_W = 0.0
REG_B = 0.0
REG_Z = 0.0
SPAR_W = 1.0
SPAR_B = 1.0
SPAR_Z = 1.0
LEARNING_RATE = 0.01
NUM_EPOCHS = 200
BATCH_SIZE = 64
GAMMA = 0.0014

In [22]:
W, B, gamma = helper.getGamma(GAMMA, PROJECTION_DIM, dataDimension,
                       NUM_PROTOTYPES, x_train)

In [23]:
protoNNObj = ProtoNN(dataDimension, PROJECTION_DIM, NUM_PROTOTYPES, numClasses,
                     gamma, W=W, B=B)
protoNNTrainer = ProtoNNTrainer(protoNNObj, REG_W, REG_B, REG_Z, SPAR_W, SPAR_B, SPAR_W,
                                LEARNING_RATE, lossType='xentropy')

Sparse training disabled.


Using x-entropy loss


In [24]:
protoNNTrainer.train(BATCH_SIZE, NUM_EPOCHS, x_train, x_test, y_train, y_test, printStep=600, valStep=10)

Epoch 0 batch 0 loss 2.302985 acc 0.046875
Epoch 0 batch 600 loss 0.276444 acc 0.875000
Epoch 0 batch 1200 loss 0.097431 acc 0.953125
Epoch 0 batch 1800 loss 0.204340 acc 0.921875
Epoch 0 batch 2400 loss 0.091853 acc 0.953125
Epoch 0 batch 3000 loss 0.227831 acc 0.906250
Epoch 1 batch 0 loss 0.187533 acc 0.906250
Epoch 1 batch 600 loss 0.140645 acc 0.937500
Epoch 1 batch 1200 loss 0.062560 acc 0.984375
Epoch 1 batch 1800 loss 0.182336 acc 0.906250
Epoch 1 batch 2400 loss 0.056413 acc 0.968750
Epoch 1 batch 3000 loss 0.171368 acc 0.906250
Epoch 2 batch 0 loss 0.140798 acc 0.921875
Epoch 2 batch 600 loss 0.095268 acc 0.953125
Epoch 2 batch 1200 loss 0.060553 acc 0.984375
Epoch 2 batch 1800 loss 0.167211 acc 0.906250
Epoch 2 batch 2400 loss 0.044242 acc 0.968750
Epoch 2 batch 3000 loss 0.155773 acc 0.906250
Epoch 3 batch 0 loss 0.101440 acc 0.953125
Epoch 3 batch 600 loss 0.050984 acc 0.984375
Epoch 3 batch 1200 loss 0.062507 acc 0.968750
Epoch 3 batch 1800 loss 0.122082 acc 0.937500
Epoc

Epoch 29 batch 1200 loss 0.038553 acc 1.000000
Epoch 29 batch 1800 loss 0.006864 acc 1.000000
Epoch 29 batch 2400 loss 0.000186 acc 1.000000
Epoch 29 batch 3000 loss 0.074564 acc 0.968750
Validation accuracy: 0.983306
Epoch 30 batch 0 loss 0.131044 acc 0.968750
Epoch 30 batch 600 loss 0.000472 acc 1.000000
Epoch 30 batch 1200 loss 0.039308 acc 0.984375
Epoch 30 batch 1800 loss 0.005047 acc 1.000000
Epoch 30 batch 2400 loss 0.000286 acc 1.000000
Epoch 30 batch 3000 loss 0.075242 acc 0.968750
Epoch 31 batch 0 loss 0.037259 acc 0.984375
Epoch 31 batch 600 loss 0.000722 acc 1.000000
Epoch 31 batch 1200 loss 0.017104 acc 1.000000
Epoch 31 batch 1800 loss 0.004719 acc 1.000000
Epoch 31 batch 2400 loss 0.000243 acc 1.000000
Epoch 31 batch 3000 loss 0.074733 acc 0.968750
Epoch 32 batch 0 loss 0.037128 acc 0.984375
Epoch 32 batch 600 loss 0.000405 acc 1.000000
Epoch 32 batch 1200 loss 0.000934 acc 1.000000
Epoch 32 batch 1800 loss 0.003921 acc 1.000000
Epoch 32 batch 2400 loss 0.000963 acc 1.00

Epoch 58 batch 1200 loss 0.000250 acc 1.000000
Epoch 58 batch 1800 loss 0.000626 acc 1.000000
Epoch 58 batch 2400 loss 0.000007 acc 1.000000
Epoch 58 batch 3000 loss 0.001221 acc 1.000000
Epoch 59 batch 0 loss 0.000418 acc 1.000000
Epoch 59 batch 600 loss 0.000127 acc 1.000000
Epoch 59 batch 1200 loss 0.000880 acc 1.000000
Epoch 59 batch 1800 loss 0.000585 acc 1.000000
Epoch 59 batch 2400 loss 0.000248 acc 1.000000
Epoch 59 batch 3000 loss 0.000580 acc 1.000000
Validation accuracy: 0.999204
Epoch 60 batch 0 loss 0.000794 acc 1.000000
Epoch 60 batch 600 loss 0.000198 acc 1.000000
Epoch 60 batch 1200 loss 0.000202 acc 1.000000
Epoch 60 batch 1800 loss 0.000116 acc 1.000000
Epoch 60 batch 2400 loss 0.000048 acc 1.000000
Epoch 60 batch 3000 loss 0.000759 acc 1.000000
Epoch 61 batch 0 loss 0.000833 acc 1.000000
Epoch 61 batch 600 loss 0.000052 acc 1.000000
Epoch 61 batch 1200 loss 0.000152 acc 1.000000
Epoch 61 batch 1800 loss 0.000143 acc 1.000000
Epoch 61 batch 2400 loss 0.000604 acc 1.00

Epoch 87 batch 1200 loss 0.000030 acc 1.000000
Epoch 87 batch 1800 loss 0.000027 acc 1.000000
Epoch 87 batch 2400 loss 0.000030 acc 1.000000
Epoch 87 batch 3000 loss 0.000647 acc 1.000000
Epoch 88 batch 0 loss 0.000270 acc 1.000000
Epoch 88 batch 600 loss 0.000160 acc 1.000000
Epoch 88 batch 1200 loss 0.000059 acc 1.000000
Epoch 88 batch 1800 loss 0.000099 acc 1.000000
Epoch 88 batch 2400 loss 0.000010 acc 1.000000
Epoch 88 batch 3000 loss 0.000222 acc 1.000000
Epoch 89 batch 0 loss 0.000459 acc 1.000000
Epoch 89 batch 600 loss 0.000351 acc 1.000000
Epoch 89 batch 1200 loss 0.000036 acc 1.000000
Epoch 89 batch 1800 loss 0.000051 acc 1.000000
Epoch 89 batch 2400 loss 0.000015 acc 1.000000
Epoch 89 batch 3000 loss 0.003012 acc 1.000000
Validation accuracy: 0.999907
Epoch 90 batch 0 loss 0.000486 acc 1.000000
Epoch 90 batch 600 loss 0.000025 acc 1.000000
Epoch 90 batch 1200 loss 0.000063 acc 1.000000
Epoch 90 batch 1800 loss 0.000036 acc 1.000000
Epoch 90 batch 2400 loss 0.000012 acc 1.00

Epoch 116 batch 0 loss 0.000156 acc 1.000000
Epoch 116 batch 600 loss 0.000088 acc 1.000000
Epoch 116 batch 1200 loss 0.000022 acc 1.000000
Epoch 116 batch 1800 loss 0.000025 acc 1.000000
Epoch 116 batch 2400 loss 0.000013 acc 1.000000
Epoch 116 batch 3000 loss 0.000623 acc 1.000000
Epoch 117 batch 0 loss 0.000213 acc 1.000000
Epoch 117 batch 600 loss 0.000120 acc 1.000000
Epoch 117 batch 1200 loss 0.000029 acc 1.000000
Epoch 117 batch 1800 loss 0.000048 acc 1.000000
Epoch 117 batch 2400 loss 0.000171 acc 1.000000
Epoch 117 batch 3000 loss 0.000340 acc 1.000000
Epoch 118 batch 0 loss 0.011722 acc 0.984375
Epoch 118 batch 600 loss 0.000133 acc 1.000000
Epoch 118 batch 1200 loss 0.000028 acc 1.000000
Epoch 118 batch 1800 loss 0.000039 acc 1.000000
Epoch 118 batch 2400 loss 0.000291 acc 1.000000
Epoch 118 batch 3000 loss 0.000337 acc 1.000000
Epoch 119 batch 0 loss 0.000041 acc 1.000000
Epoch 119 batch 600 loss 0.000134 acc 1.000000
Epoch 119 batch 1200 loss 0.000038 acc 1.000000
Epoch 11

Epoch 144 batch 1800 loss 0.000043 acc 1.000000
Epoch 144 batch 2400 loss 0.000234 acc 1.000000
Epoch 144 batch 3000 loss 0.000890 acc 1.000000
Epoch 145 batch 0 loss 0.000094 acc 1.000000
Epoch 145 batch 600 loss 0.000630 acc 1.000000
Epoch 145 batch 1200 loss 0.000037 acc 1.000000
Epoch 145 batch 1800 loss 0.000023 acc 1.000000
Epoch 145 batch 2400 loss 0.000011 acc 1.000000
Epoch 145 batch 3000 loss 0.000943 acc 1.000000
Epoch 146 batch 0 loss 0.000032 acc 1.000000
Epoch 146 batch 600 loss 0.000023 acc 1.000000
Epoch 146 batch 1200 loss 0.000066 acc 1.000000
Epoch 146 batch 1800 loss 0.000091 acc 1.000000
Epoch 146 batch 2400 loss 0.002343 acc 1.000000
Epoch 146 batch 3000 loss 0.000104 acc 1.000000
Epoch 147 batch 0 loss 0.000075 acc 1.000000
Epoch 147 batch 600 loss 0.000333 acc 1.000000
Epoch 147 batch 1200 loss 0.000012 acc 1.000000
Epoch 147 batch 1800 loss 0.000041 acc 1.000000
Epoch 147 batch 2400 loss 0.000048 acc 1.000000
Epoch 147 batch 3000 loss 0.001208 acc 1.000000
Epoc

Epoch 173 batch 0 loss 0.000035 acc 1.000000
Epoch 173 batch 600 loss 0.000356 acc 1.000000
Epoch 173 batch 1200 loss 0.000016 acc 1.000000
Epoch 173 batch 1800 loss 0.000024 acc 1.000000
Epoch 173 batch 2400 loss 0.000183 acc 1.000000
Epoch 173 batch 3000 loss 0.000717 acc 1.000000
Epoch 174 batch 0 loss 0.000154 acc 1.000000
Epoch 174 batch 600 loss 0.000246 acc 1.000000
Epoch 174 batch 1200 loss 0.000016 acc 1.000000
Epoch 174 batch 1800 loss 0.000017 acc 1.000000
Epoch 174 batch 2400 loss 0.000026 acc 1.000000
Epoch 174 batch 3000 loss 0.000709 acc 1.000000
Epoch 175 batch 0 loss 0.000088 acc 1.000000
Epoch 175 batch 600 loss 0.000151 acc 1.000000
Epoch 175 batch 1200 loss 0.000010 acc 1.000000
Epoch 175 batch 1800 loss 0.000987 acc 1.000000
Epoch 175 batch 2400 loss 0.000033 acc 1.000000
Epoch 175 batch 3000 loss 0.000259 acc 1.000000
Epoch 176 batch 0 loss 0.000096 acc 1.000000
Epoch 176 batch 600 loss 0.000461 acc 1.000000
Epoch 176 batch 1200 loss 0.000004 acc 1.000000
Epoch 17

## Evaluation

In [25]:
 x_, y_= torch.Tensor(x_test), torch.Tensor(y_test)
logits = protoNNObj.forward(x_)
_, predictions = torch.max(logits, dim=1)
_, target = torch.max(y_, dim=1)
acc, count = protoNNTrainer.accuracy(predictions, target)
W, B, Z, gamma  = protoNNObj.getModelMatrices()
matrixList = [W, B, Z]
matrixList = [x.detach().numpy() for x in matrixList]
sparcityList = [SPAR_W, SPAR_B, SPAR_Z]
nnz, size, sparse = helper.getModelSize(matrixList, sparcityList)
print("Final test accuracy", acc)
print("Model size constraint (Bytes): ", size)
print("Number of non-zeros: ", nnz)
nnz, size, sparse = helper.getModelSize(matrixList, sparcityList,
                                        expected=False)
print("Actual model size: ", size)
print("Actual non-zeros: ", nnz)

Final test accuracy tensor(1., dtype=torch.float64)
Model size constraint (Bytes):  6880
Number of non-zeros:  1720
Actual model size:  6880
Actual non-zeros:  1720


In [26]:
W = W.detach().numpy()
B = B.detach().numpy()
Z = Z.detach().numpy()
W = np.transpose(W)
B = np.transpose(B)
Z = np.transpose(Z)
print(W.shape)
print(B.shape)
print(Z.shape)
np.savetxt("W", W, fmt="%f", delimiter=",")
np.savetxt("B", B, fmt="%f", delimiter=",")
np.savetxt("Z", Z, fmt="%f", delimiter=",")

(10, 124)
(24, 10)
(24, 10)


In [27]:
gamma

0.0014